# Tarea 4: Aprendizaje por Refuerzo

#Objetivos

 - Entender conceptos y técnicas básicas de Reinforcement Learning.

 -Implementar RL mediante políticas en entornos controlados simples.

## 1.	El Problema
Un poste está unido mediante una articulación no accionada a un carro, que se mueve a lo largo de una vía sin fricción. El péndulo se coloca en posición vertical sobre el carro y el objetivo es equilibrar el poste aplicando fuerzas en la dirección izquierda y derecha sobre el carro


![Cartpole environment](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

###1.1.	Desafío
Entrenar un agente de mediante aprendizaje por refuerzo, ajustando sus hiper parámetros, que sea capaz de aprender una política que le permita mantener el equilibrio por más de 40 segundos.

###1.2.	Salida
Como resultado, se debe entregar el conjunto de hiper parámetros utilizados, justificando su elección, así como un video del carro sosteniendo el equilibrio.



## Dependencias!

Si no ha instalado las siguientes dependencias, ejecute:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

SyntaxError: invalid syntax (reverb.py, line 64)

In [ ]:
# Configure una pantalla virtual para representar entornos de OpenAI gym.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION #debe dar version superior a 2.14.0

## Hiperparámetros

In [ ]:
num_iterations = 1 # @param {type:"integer"}

initial_collect_steps = 1  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 1  # @param {type:"integer"}

batch_size = 1  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 1  # @param {type:"integer"}

num_eval_episodes = 1  # @param {type:"integer"}
eval_interval = 1  # @param {type:"integer"}

## Ambiente

En el Aprendizaje por Refuerzo (RL), un entorno representa la tarea o problema a resolver.

Cargue el entorno CartPole desde la suite OpenAI Gym.

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

Puede renderizar este entorno para ver cómo se ve. Un poste que se balancea libremente está sujeto a un carro.

El objetivo es mover el carro hacia la derecha o hacia la izquierda para mantener el poste apuntando hacia arriba.

In [ ]:
env.reset()
PIL.Image.fromarray(env.render())

El método `environment.step` toma una `acción` en el entorno y devuelve una tupla `TimeStep` que contiene la siguiente observación del entorno y la recompensa por la acción.

El método `time_step_spec()` devuelve la especificación de la tupla `TimeStep`. Su atributo "observación" muestra la forma de las observaciones, los tipos de datos y los rangos de valores permitidos. El atributo "recompensa" muestra los mismos detalles para la recompensa.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

El método `action_spec()` devuelve la forma, los tipos de datos y los valores permitidos de acciones válidas.

In [ ]:
print('Action Spec:')
print(env.action_spec())

En el entorno de Cartpole:

- `observación` es una matriz de 4 flotantes:
     - la posición y velocidad del carro
     - la posición angular y la velocidad del polo
- `recompensa` es un valor flotante escalar
- `acción` es un número entero escalar con sólo dos valores posibles:
     - `0` — "mover a la izquierda"
     - `1` — "mover a la derecha"

In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Por lo general, se crean instancias de dos entornos: uno para capacitación y otro para evaluación.

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

El entorno Cartpole, como la mayoría de los entornos, está escrito en Python puro. Esto se convierte a TensorFlow usando el contenedor `TFPyEnvironment`.

La API del entorno original utiliza matrices Numpy. El "TFPyEnvironment" los convierte en "Tensores" para hacerlos compatibles con los agentes y políticas de Tensorflow.

In [ ]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

El algoritmo utilizado para resolver un problema de RL está representado por un "Agente". TF-Agents proporciona implementaciones estándar de una variedad de "Agentes".

El agente DQN se puede utilizar en cualquier entorno que tenga un espacio de acción discreto.

En el corazón de un agente DQN se encuentra una "QNetwork", un modelo de red neuronal que puede aprender a predecir "QValues" (rendimientos esperados) para todas las acciones, dada una observación del entorno.

Usaremos `tf_agents.networks.` para crear una `QNetwork`. La red constará de una secuencia de capas `tf.keras.layers.Dense`, donde la capa final tendrá 1 salida para cada acción posible.

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

Ahora use `tf_agents.agents.dqn.dqn_agent` para crear una instancia de un `DqnAgent`. Además de `time_step_spec`, `action_spec` y QNetwork, el constructor del agente también requiere un optimizador (en este caso, `AdamOptimizer`), una función de pérdida y un contador de pasos de números enteros.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Políticas

Una política define la forma en que actúa un agente en un entorno.

Normalmente, el objetivo del aprendizaje por refuerzo es entrenar el modelo subyacente hasta que la política produzca el resultado deseado.

Objetivos:

- El resultado deseado es mantener el poste en equilibrio sobre el carro.
- La política devuelve una acción (izquierda o derecha) para cada observación de `time_step`.

Los agentes contienen dos políticas:

- `agent.policy`: la política principal que se utiliza para la evaluación y la implementación.
- `agent.collect_policy`: una segunda política que se utiliza para la recopilación de datos.

In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

## Métricas y Evaluación

La métrica más común utilizada para evaluar una póliza es el rendimiento promedio.

El retorno es la suma de las recompensas obtenidas al ejecutar una póliza en un entorno para un episodio.

Se ejecutan varios episodios, lo que genera un rendimiento medio.

La siguiente función calcula el rendimiento promedio de una póliza, dada la política, el entorno y una cantidad de episodios.

In [ ]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

## Búfer de reproducción

Para realizar un seguimiento de los datos recopilados del entorno, utilizaremos [Reverb](https://deepmind.com/research/open-source/Reverb), un sistema de reproducción eficiente, extensible y fácil de usar.

Almacena datos de experiencia cuando recopilamos trayectorias y se consume durante el entrenamiento.

Este búfer de reproducción se construye utilizando especificaciones que describen los tensores que se almacenarán, que se pueden obtener del agente mediante agent.collect_data_spec.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

Para la mayoría de los agentes, `collect_data_spec` es una `named tuple`  llamada `Trayectoria`, que contiene las especificaciones para observaciones, acciones, recompensas y otros elementos.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## Entrenando al agente (Implemente su loop de entrenamiento aqui!)

Deben suceder dos cosas durante el ciclo de entrenamiento:

- recopilar datos del medio ambiente
- utilizar esos datos para entrenar las redes neuronales del agente

Este ejemplo también evalúa periódicamente la política e imprime la puntuación actual.

Lo siguiente tardará 5-10 minutos en ejecutarse.

In [ ]:
#Implementar Aqui la politica de entrenamiento
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.agents.dqn.dqn_agent import DqnAgent

# Asegúrate de que todas las importaciones necesarias estén aquí
import tensorflow as tf



# ... (resto de tu configuración y definiciones previas)

# Configurar el buffer de repetición
replay_buffer = TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

# ... (code for agent configuration and functions)


# Bucle de entrenamiento
for _ in range(num_iterations):
    # Recopilar unos pocos pasos usando collect_policy y guardar en el buffer
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent.collect_policy)

    # Muestrear un lote de datos del buffer y actualizar la red del agente
    # Asegúrate de que 'iterator' esté definido y configurado correctamente aquí
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))


## Visualization


### Gráficos

Utilice `matplotlib.pyplot` para registrar cómo mejoró la política durante el entrenamiento.

Una iteración de `Cartpole-v0` consta de 200 pasos de tiempo. El entorno ofrece una recompensa de "+1" por cada paso que el poste se mantenga arriba, por lo que el retorno máximo para un episodio es 200. Los gráficos muestran que el retorno aumenta hacia ese máximo cada vez que se evalúa durante el entrenamiento. (Puede ser un poco inestable y no aumentar de forma monótona cada vez).

In [ ]:
iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Videos

Los gráficos son bonitos.

Pero lo más emocionante es ver a un agente realizando una tarea en un entorno.

Primero, cree una función para insertar videos en el notebook.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Ahora repite algunos episodios del juego Cartpole con el agente. El entorno Python subyacente (el que está "dentro" del contenedor del entorno TensorFlow) proporciona un método `render()`, que genera una imagen del estado del entorno. Estos se pueden recopilar en un vídeo.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

POr diversion, compara al agente entrenado (arriba) con un agente que se mueve al azar. (No funciona tan bien).

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())
create_policy_eval_video(random_policy, "random-agent")